In [0]:
import urllib.request 
from urllib.parse import quote 
from bs4 import BeautifulSoup
import json
import re
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#Source: https://codezup.com/web-scraping-word-meaning-dictionary-python-beautifulsoup/

In [4]:
import json
with open('/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/freqlist.json') as json_file:
     wordsData = json.load(json_file)
wordList = list(wordsData.keys())
len(wordList)

14143

In [0]:
#The link format for vdict Viet-Fran is: https://vdict.com/m%E1%BB%99t,4,0,0.html

In [0]:
#Converting IRI to ASCII:
#https://stackoverflow.com/questions/4389572/how-to-fetch-a-non-ascii-url-with-python-urlopen

In [0]:
def findTranslation(word):
    print("Finding translation for " + word)
    url = "https://vdict.com/" + quote(word) + ",4,0,0.html" #resolving the IRI issue
    
    try: #make sure the link is working
        source = urllib.request.urlopen(url)
    except:
        print("Link broken for " + word)
        return "N/A"
    
    soup = BeautifulSoup(source, 'lxml')

    #make sure the word is in the dictionary
    if (soup.find("div", id = 'result-contents') is None):
        print("Not in the dictionary " + word)
        return "N/A"

    translations = {}

    #Getting the first translation
    curTag = soup.find("ul", class_= "list1")

    if (curTag.find('b') is None): #empty translation page
        print("Empty translation page " + word)
        return "N/A"

    #storing the number of translations a word has
    numTrans = 1

    #continue while we still have more translations
    while(curTag != '\n'): 
        temp = {} #temp dict to store each translation of a word

        #getting the raw translation
        if (curTag.find('b').string is not None): #empty translation cell
            temp['translation'] = re.split('[,;]', curTag.find('b').string)

            #retrieving the examples of each of the translations
            try:
                examples = curTag.findAll("ul", class_= "list2")
                temp2 = {} #another dict to store each examples of a translation

                numExamples = len(list(examples))
                temp2['nums'] = numExamples

                for j in range(0, numExamples):
                    temp2['context ' + str(j+1)] = examples[j].find(class_ = "example-original").string

                    #retrieving the usage of each examples
                    try: 
                        tempString = str(examples[j].find("li")) #extracting the usage
                        temp2['usage ' + str(j+1)] = tempString[tempString.find("<br/>")+5:-5]
                    except:
                        temp2['usage ' + str(j+1)] = "N/A"

                temp['examples'] = temp2
            except:
                temp['examples'] = "N/A"

            translations[numTrans] = temp #adding the translation to the dictionary

            numTrans += 1
        #go to next tag  
        curTag = curTag.next_sibling
    
    translations["nums"] = numTrans - 1
        
    return translations

In [0]:
url = "https://vdict.com/" + quote("có") + ",4,0,0.html" #resolving the IRI issue

try:
    source = urllib.request.urlopen(url)
except:
    print( "Link broken")
soup = BeautifulSoup(source, 'lxml')

In [9]:
soup.find("ul", class_= "list1").next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling

'\n'

In [10]:
containers = soup.findAll("ul", class_= "list1")
containers

[<ul class="list1"><li><b>avoir; posséder</b><ul class="list2"><li><span class="example-original">Tôi có nói câu đó </span><br/>j'ai dit cette phrase</li></ul><ul class="list2"><li><span class="example-original">Có tiền </span><br/>avoir de l'argent</li></ul><ul class="list2"><li><span class="example-original">Có một cái nhà </span><br/>posséder une maison</li></ul></li></ul>,
 <ul class="list1"><li><b>exister; se trouver</b><ul class="list2"><li><span class="example-original">Cái tục đó còn đó </span><br/>cet usage existe encore</li></ul><ul class="list2"><li><span class="example-original">Loài chim đó không có ở Việt Nam </span><br/>cette espèce d'oiseau n'existe pas au Viet-Nam</li></ul><ul class="list2"><li><span class="example-original">Tên anh không có trong danh sách </span><br/>ton nom ne se trouve pas sur la liste</li></ul></li></ul>,
 <ul class="list1"><li><b>disposer de</b><ul class="list2"><li><span class="example-original">ông ấy có một cái xe </span><br/>il dispose d'une 

In [11]:
meanings = [item.find('b').string for item in containers[0:4]] #word translation
meanings

['avoir; posséder', 'exister; se trouver', 'disposer de', 'être présent']

In [12]:
examples = containers[0].findAll("ul", class_= "list2")
temp = str(examples[1].find("li"))
temp[temp.find("<br/>")+5:-5]

"avoir de l'argent"

In [0]:
dictionary = {}

In [14]:
start_time = time.time()
for word in wordList:
    dictionary[word] = findTranslation(word)
    time.sleep(1) #wait for 1 sec between each query
print("--- %s seconds ---" % (time.time() - start_time))

Streaming output truncated to the last 5000 lines.
Finding translation for andromakhe
Not in the dictionary andromakhe
Finding translation for briseis
Not in the dictionary briseis
Finding translation for aiolian
Not in the dictionary aiolian
Finding translation for korinth
Not in the dictionary korinth
Finding translation for kalchas
Not in the dictionary kalchas
Finding translation for antenor
Not in the dictionary antenor
Finding translation for alexandros
Not in the dictionary alexandros
Finding translation for euneos
Not in the dictionary euneos
Finding translation for sarpedon
Not in the dictionary sarpedon
Finding translation for aineias
Not in the dictionary aineias
Finding translation for pandaros
Not in the dictionary pandaros
Finding translation for meleagros
Not in the dictionary meleagros
Finding translation for bellerophontes
Not in the dictionary bellerophontes
Finding translation for laomedon
Not in the dictionary laomedon
Finding translation for ida
Not in the dictiona

In [15]:
dictionary

{'thông': {1: {'examples': {'context 1': 'Thông hai lá ',
    'context 2': 'Thông ba lá ',
    'context 3': 'Thông nhựa ',
    'nums': 3,
    'usage 1': 'pin de Merkus',
    'usage 2': 'pin khasya ;',
    'usage 3': 'pin résineux'},
   'translation': ['(bot.) pin.']},
  2: {'examples': {'context 1': 'Thông ngôn ',
    'context 2': 'Thông ngôn lại ',
    'nums': 2,
    'usage 1': 'interprète',
    'usage 2': 'secrétaire des bureaux de district'},
   'translation': ['(từ cũ ',
    ' nghĩa cũ) secrétaire des bureaux de province']},
  3: {'examples': {'context 1': 'Buồng này thông sang buồng bên cạnh ',
    'nums': 1,
    'usage 1': 'cette chambre communique avec la chambre voisine.'},
   'translation': ['communiquer']},
  4: {'examples': {'context 1': 'Thông ống dẫn nước ',
    'context 2': 'thông cống ',
    'nums': 2,
    'usage 1': "désengorger une conduite d'eau",
    'usage 2': 'dégorger un égout.'},
   'translation': ['désengorger ', ' dégorger']},
  5: {'examples': {'context 1': 'T

In [0]:
from google.colab import files
with open('vdict VI-FR.json', 'w') as outfile:
    json.dump(dictionary, outfile)
files.download('vdict VI-FR.json')

In [0]:
findTranslation("thượng")